## Monte Carlo

**Any method which solves a problem by generating suitable random numbers, and observing that fraction of numbers obeying some property or properties, can be classified as a Monte Carlo method.**

Let’s do a fun exercise where we will try to find out the value of pi using pen and paper. Let’s draw a square of unit length and draw a quarter circle with unit length radius. Now, we have a helper bot C3PO with us. It is tasked with putting as many dots as possible on the square randomly 3,000 times, resulting in the following figure:

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

C3PO needs to count each time it puts a dot inside a circle. So, the value of pi will be given by:

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

where N is the number of times a dot was put inside the circle. As you can see, we did not do anything except count the random dots that fall inside the circle and then took a ratio to approximate the value of pi.

### Monte Carlo Reinforcement Learning
The Monte Carlo method for reinforcement learning learns directly from episodes of experience without any prior knowledge of MDP transitions. Here, the random component is the return or reward.

One caveat is that it can only be applied to episodic MDPs. Its fair to ask why, at this point. The reason is that the episode has to terminate before we can calculate any returns. Here, we don’t do an update after every action, but rather after every episode. It uses the simplest idea – the value is the mean return of all sample trajectories for each state.

Recalling the idea from multi-armed bandits discussed in this article, every state is a separate multi-armed bandit problem and the idea is to behave optimally for all multi-armed bandits at once.

Similar to dynamic programming, there is a policy evaluation (finding the value function for a given random policy) and policy improvement step (finding the optimum policy). We will cover both these steps in the next two sections.

 

### Monte Carlo Policy Evaluation
The goal here, again, is to learn the value function vpi(s) from episodes of experience under a policy pi. Recall that the return is the total discounted reward:

$$S_{1}, A_{1}, R_{2}, ….S_{k} ~ pi$$

Also recall that the value function is the expected return:
![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)


We know that we can estimate any expected value simply by adding up samples and dividing by the total number of samples:
![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)


i – Episode index
s – Index of state
The question is how do we get these sample returns? For that, we need to play a bunch of episodes and generate them.

For every episode we play, we’ll have a sequence of states and rewards. And from these rewards, we can calculate the return by definition, which is just the sum of all future rewards.

**First Visit Monte Carlo:** Average returns only for first time s is visited in an episode.

Here’s a step-by-step view of how the algorithm works:

1. Initialize the policy, state-value function
2. Start by generating an episode according to the current policy
    1. Keep track of the states encountered through that episode
3. Select a state in 2.1
    1. Add to a list the return received after first occurrence of this state
    2. Average over all returns
    3. Set the value of the state as that computed average
4. Repeat step 3
5. Repeat 2-4 until satisfied
**Every visit Monte Carlo:** Average returns for every time s is visited in an episode.

For this algorithm, we just change step #3.1 to ‘Add to a list the return received after every occurrence of this state’.

Let’s consider a simple example to further understand this concept. Suppose there’s an environment where we have 2 states – A and B. Let’s say we observed 2 sample episodes:

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

A+3 => A indicates a transition from state A to state A, with a reward +3. Let’s find out the value function using both methods:

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)


### Incremental Mean
It is convenient to convert the mean return into an incremental update so that the mean can be updated with each episode and we can understand the progress made with each episode. We already learnt this when solving the multi-armed bandit problem.

We update v(s) incrementally after episodes. For each state St, with return Gt:

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

In non-stationary problems, it can be useful to track a running mean, i.e., forget old episodes:

$$V(S_{t}) ← V(S_{t}) + α (G_{t} − V(S_{t}))$$

 

### Monte Carlo Control
Similar to dynamic programming, once we have the value function for a random policy, the important task that still remains is that of finding the optimal policy using Monte Carlo.

Recall that the formula for policy improvement in DP required the model of the environment as shown in the following equation:

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

This equation finds out the optimal policy by finding actions that maximize the sum of rewards. However, a major caveat here is that it uses transition probabilities, which is not known in the case of model-free learning.

Since we do not know the state transition probabilities p(s’,r/s,a), we can’t do a look-ahead search like DP. Hence, all the information is obtained via experience of playing the game or exploring the environment.

Policy improvement is done by making the policy greedy with respect to the current value function. In this case, we have an action-value function, and therefore no model is needed to construct the greedy policy.

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

A greedy policy (like the above mentioned one) will always favor a certain action if most actions are not explored properly. There are two solutions for this:

#### Monte Carlo with exploring starts

All the state action pairs have non-zero probability of being the starting pair, in this algorithm. This will ensure each episode which is played will take the agent to new states and hence, there is more exploration of the environment.

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

#### Monte Carlo with epsilon-Soft

What if there is a single start point for an environment (for example, a game of chess)? Exploring starts is not the right option in such cases. Recall here that in a multi-armed bandit problem, we discussed the epsilon-greedy approach.

Simplest idea for ensuring continual exploration all actions are tried with non-zero probability 1 – epsilon choose the action which maximises the action value function and with probability epsilon choose an action at random.

![Title](../docs/images/ARS/MethodOfFiniteDifference_Equation.png)

Now that we understand the basics of Monte Carlo Control and Prediction, let’s implement the algorithm in Python. We will import the frozen lake environment from the popular OpenAI Gym toolkit.



## Importing libraries

In [ ]:
# Importing the libraries
import gym
import numpy as np
import operator
from IPython.display import clear_output
from time import sleep
import random
import itertools
import tqdm  ## Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable)
## for i in tqdm(range(10000)):

tqdm.monitor_interval = 0

In [ ]:
## Function for Random Policy

def create_random_policy(env):
    policy = {}
    for key in range(0, env.observation_space.n):
        current_end = 0
        p = {}
        for action in range(0, env.action_space.n):
            p[action] = 1 / env.action_space.n
        policy[key] = p
    return policy

## Dictionary for storing the state action value
def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
         Q[key] = {a: 0.0 for a in range(0, env.action_space.n)}
    return Q


In [ ]:
## Function to play episode

def run_game(env, policy, display=True):
    env.reset()
    episode = []
    finished = False

    while not finished:
        s = env.env.s
        if display:
            clear_output(True)
            env.render()
            sleep(1)

        timestep = []
        timestep.append(s)
        n = random.uniform(0, sum(policy[s].values()))
        top_range = 0
        for prob in policy[s].items():
            top_range += prob[1]
            if n < top_range:
                action = prob[0]
                break 
        state, reward, finished, info = env.step(action)
        timestep.append(action)
        timestep.append(reward)

        episode.append(timestep)

    if display:
        clear_output(True)
        env.render()
        sleep(1)
    return episode

In [ ]:
## Function to test policy and print win percentage
def test_policy(policy, env):
    wins = 0
    r = 100
    for i in range(r):
        w = run_game(env, policy, display=False)[-1][-1]
        if w == 1:
            wins += 1
    return wins / r

In [ ]:
## First Visit Monte Carlo Prediction and Control
def monte_carlo_e_soft(env, episodes=100, policy=None, epsilon=0.01):
    if not policy:
        policy = create_random_policy(env)  # Create an empty dictionary to store state action values    
    Q = create_state_action_dictionary(env, policy) # Empty dictionary for storing rewards for each state-action pair
    returns = {} # 3.
    
    for _ in range(episodes): # Looping through episodes
        G = 0 # Store cumulative reward in G (initialized at 0)
        episode = run_game(env=env, policy=policy, display=False) # Store state, action and value respectively 
        
        # for loop through reversed indices of episode array. 
        # The logic behind it being reversed is that the eventual reward would be at the end. 
        # So we have to go back from the last timestep to the first one propagating result from the future.
        
        for i in reversed(range(0, len(episode))):   
            s_t, a_t, r_t = episode[i] 
            state_action = (s_t, a_t)
            G += r_t # Increment total reward by reward on current timestep
            
            if not state_action in [(x[0], x[1]) for x in episode[0:i]]: # 
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]   
                    
                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                
                Q_list = list(map(lambda x: x[1], Q[s_t].items())) # Finding the action with maximum value
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                max_Q = random.choice(indices)
                
                A_star = max_Q # 14.
                
                for a in policy[s_t].items(): # Update action probability for s_t in policy
                    if a[0] == A_star:
                        policy[s_t][a[0]] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                    else:
                        policy[s_t][a[0]] = (epsilon / abs(sum(policy[s_t].values())))

    return policy

In [ ]:
## Running the Algorithm
env = gym.make('FrozenLake8x8-v0')
policy = monte_carlo_e_soft(env, episodes=5000)
test_policy(policy, env)

## Setting Configuration

## Normalizing the  states

## Building AI

## Additional Functions

## Main Function